In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, LSTM, GRU, Dropout

In [ ]:
tr_df = pd.read_csv('/kaggle/input/time-series-practice-dataset/train.csv')

In [ ]:
tr_df.head()

In [ ]:
tr_df.info()

In [ ]:
tr_df['Date'] = pd.to_datetime(tr_df['Date'])

In [ ]:
tr_df.info()

In [ ]:
tr_df['store'].value_counts()

In [ ]:
tr_df['product'].value_counts()

## We will deal with each store indivedually

# Store 1

In [ ]:
tr_df.set_index('Date', inplace= True)

In [ ]:
tr_df

## 1st Product

In [ ]:
# Store 1 with product 1
s1p1 = tr_df[(tr_df['store'] == 0) & (tr_df['product'] == 0)]
s1p1

In [ ]:
plt.figure(figsize = (15, 6))
plt.plot(s1p1.index, s1p1['number_sold'])
plt.xlabel('Date', {'fontsize': 12})
plt.ylabel('Number Sold', {'fontsize': 12})
plt.title('Number Sold Of Product 1 In The First Store')
plt.show()

_"number_sold"_ is our target

In [ ]:
n_cols = 1
dataset = s1p1["number_sold"]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

Let's rescale the values to make it easy for the model

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

Splitting data to train and test data to make training [75% to Train , 25% to Test]

In [ ]:
train_size = int(len(data) * 0.75)
test_size = len(data) - train_size
print("Train Size :",train_size,"Test Size :",test_size)

In [ ]:
train_data = scaled_data[0:train_size, :]
train_data.shape

LSTM takes a 3D input (num_samples, num_timesteps, num_features), so we will split data according that.

In [ ]:
# Creating a Training set with 60 time-steps and 1 target
x_train = []
y_train = []
time_steps = 60
n_cols = 1

for i in range(time_steps, len(train_data)):
    x_train.append(train_data[i-time_steps:i, :n_cols])
    y_train.append(train_data[i, :n_cols])
    if i<=time_steps:
        print('X_train: ', x_train)
        print('y_train:' , y_train)

In [ ]:
# Convert to numpy array
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# Reshaping the input to (n_samples, time_steps, n_feature)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_cols))

In [ ]:
x_train.shape , y_train.shape

# Model Structure

In [ ]:
model = Sequential([
    LSTM(50, return_sequences= True, input_shape= (x_train.shape[1], n_cols)),
    LSTM(64, return_sequences= False),
    Dense(32),
    Dense(16),
    Dense(n_cols)
])

model.compile(optimizer= 'adam', loss= 'mse' , metrics= "mean_absolute_error")

In [ ]:
model.summary()

I train the model with train data for 100 epoch and batch_size = 32.

To avoid overfitting, I set an _EarlyStoping_ to stop training when "val_loss" has not improved after 10 epochs _(patience = 10).

In [ ]:
# Fitting the LSTM to the Training set
callbacks = [EarlyStopping(monitor= 'loss', patience= 10 , restore_best_weights= True)]
history = model.fit(x_train, y_train, epochs= 100, batch_size= 32 , callbacks= callbacks )

# Training Evaluation

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history["loss"])
plt.plot(history.history["mean_absolute_error"])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

Let's split test data to the same format (num_samples, num_timesteps, num_features)

In [ ]:
# Creating a testing set with 60 time-steps and 1 output
time_steps = 60
test_data = scaled_data[train_size - time_steps:, :]

x_test = []
y_test = []
n_cols = 1

for i in range(time_steps, len(test_data)):
    x_test.append(test_data[i-time_steps:i, 0:n_cols])
    y_test.append(test_data[i, 0:n_cols])
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], n_cols))

In [ ]:
x_test.shape , y_test.shape

### Get Prediction

In [ ]:
predictions = model.predict(x_test)

In [ ]:
#inverse predictions scaling
predictions = scaler.inverse_transform(predictions)
predictions.shape

# Prediction Evaluation With Root Mean Square Error

In [ ]:
#inverse y_test scaling
y_test = scaler.inverse_transform(y_test)

RMSE = np.sqrt(np.mean( y_test - predictions )**2).round(2)
RMSE

In [ ]:
preds_acts = pd.DataFrame(data={'Predictions':predictions.flatten(), 'Actuals':y_test.flatten()})
preds_acts

In [ ]:
train = dataset.iloc[:train_size , 0:1]
test = dataset.iloc[train_size: , 0:1]
test['Predictions'] = predictions

plt.figure(figsize= (16, 6))
plt.title('Number Sold Prediction', fontsize= 18)
plt.xlabel('Date', fontsize= 18)
plt.ylabel('Number Sold', fontsize= 18)
plt.plot(train['number_sold'], linewidth= 3)
plt.plot(test['number_sold'], linewidth= 3)
plt.plot(test["Predictions"], linewidth= 3)
plt.legend(['Train', 'Test', 'Predictions'])

# Forecast Next 60 Days

In [ ]:
from datetime import timedelta

Function to store next value with old value

In [ ]:
def insert_end(Xin, new_input):
    timestep = 60
    for i in range(timestep - 1):
        Xin[:, i, :] = Xin[:, i+1, :]
    Xin[:, timestep - 1, :] = new_input
    return Xin

create time and forecasted data

In [ ]:
future = 30 * 2
forcast = []
Xin = x_test[-1 :, :, :]
time = []
for i in range(0, future):
    out = model.predict(Xin, batch_size=5)
    forcast.append(out[0, 0]) 
    print(forcast)
    Xin = insert_end(Xin, out[0, 0]) 
    time.append(pd.to_datetime(s1p1.index[-1]) + timedelta(days=i))

In [ ]:
time

create _forecasted dataframe_

In [ ]:
forcasted_output = np.asanyarray(forcast)   
forcasted_output = forcasted_output.reshape(-1, 1) 
forcasted_output = scaler.inverse_transform(forcasted_output) 

In [ ]:
forcasted_output = pd.DataFrame(forcasted_output)
date = pd.DataFrame(time)
df_result = pd.concat([date,forcasted_output], axis=1)
df_result.columns = "Date", "Forecasted"

In [ ]:
df_result

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Number Sold Forecasting For Next 60 Days')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Number Sold' ,fontsize=18)
plt.plot(s1p1['number_sold'])
plt.plot(df_result.set_index('Date')[['Forecasted']])

## 2nd Product
We will repeat the previous steps but in the data of product 2

In [ ]:
# Store 1 with product 2
s1p2 = tr_df[(tr_df['store'] == 0) & (tr_df['product'] == 1)]
s1p2

In [ ]:
plt.figure(figsize = (15, 6))
plt.plot(s1p2.index, s1p2['number_sold'])
plt.xlabel('Date', {'fontsize': 12})
plt.ylabel('Number Sold', {'fontsize': 12})
plt.title('Number Sold Of Product 2 In The First Store')
plt.show()

In [ ]:
n_cols = 1
dataset = s1p2["number_sold"]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

In [ ]:
# 75% to Train , 25% to Test
train_size = int(len(data) * 0.75)
test_size = len(data) - train_size
print("Train Size :",train_size,"Test Size :",test_size)

In [ ]:
train_data = scaled_data[0:train_size, :]
train_data.shape

In [ ]:
# Creating a Training set with 60 time-steps
x_train = []
y_train = []
time_steps = 60
n_cols = 1

for i in range(time_steps, len(train_data)):
    x_train.append(train_data[i-time_steps:i, :n_cols])
    y_train.append(train_data[i, :n_cols])
    if i<=time_steps:
        print('X_train: ', x_train)
        print('y_train:' , y_train)

In [ ]:
# Convert to numpy array
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshaping the input to (n_samples, time_steps, n_feature)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_cols))
x_train.shape , y_train.shape

In [ ]:
model = Sequential([
    LSTM(50, return_sequences= True, input_shape= (x_train.shape[1], n_cols)),
    LSTM(64, return_sequences= False),
    Dense(32),
    Dense(16),
    Dense(n_cols)
])

model.compile(optimizer= 'adam', loss= 'mse' , metrics= "mean_absolute_error")
model.summary()

In [ ]:
# Fitting the LSTM to the Training set
callbacks = [EarlyStopping(monitor= 'loss', patience= 10 , restore_best_weights= True)]
history = model.fit(x_train, y_train, epochs= 100, batch_size= 32 , callbacks= callbacks )

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history["loss"])
plt.plot(history.history["mean_absolute_error"])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
# Creating a testing set with 60 time-steps and 1 output
time_steps = 60
test_data = scaled_data[train_size - time_steps:, :]

x_test = []
y_test = []
n_cols = 1

for i in range(time_steps, len(test_data)):
    x_test.append(test_data[i-time_steps:i, 0:n_cols])
    y_test.append(test_data[i, 0:n_cols])
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], n_cols))

x_test.shape , y_test.shape

In [ ]:
# Get Prediction
predictions = model.predict(x_test)

#inverse predictions scaling
predictions = scaler.inverse_transform(predictions)
predictions.shape

In [ ]:
#inverse y_test scaling
y_test = scaler.inverse_transform(y_test)

RMSE = np.sqrt(np.mean( y_test - predictions )**2).round(2)
RMSE

In [ ]:
preds_acts = pd.DataFrame(data={'Predictions':predictions.flatten(), 'Actuals':y_test.flatten()})
preds_acts

In [ ]:
train = dataset.iloc[:train_size , 0:1]
test = dataset.iloc[train_size: , 0:1]
test['Predictions'] = predictions

plt.figure(figsize= (16, 6))
plt.title('Number Sold Prediction', fontsize= 18)
plt.xlabel('Date', fontsize= 18)
plt.ylabel('Number Sold', fontsize= 18)
plt.plot(train['number_sold'], linewidth= 3)
plt.plot(test['number_sold'], linewidth= 3)
plt.plot(test["Predictions"], linewidth= 3)
plt.legend(['Train', 'Test', 'Predictions'])

In [ ]:
from datetime import timedelta

In [ ]:
def insert_end(Xin, new_input):
    timestep = 60
    for i in range(timestep - 1):
        Xin[:, i, :] = Xin[:, i+1, :]
    Xin[:, timestep - 1, :] = new_input
    return Xin

In [ ]:
future = 30 * 2
forcast = []
Xin = x_test[-1 :, :, :]
time = []
for i in range(0, future):
    out = model.predict(Xin, batch_size=5)
    forcast.append(out[0, 0]) 
    print(forcast)
    Xin = insert_end(Xin, out[0, 0]) 
    time.append(pd.to_datetime(s1p1.index[-1]) + timedelta(days=i))

In [ ]:
time

In [ ]:
forcasted_output = np.asanyarray(forcast)   
forcasted_output = forcasted_output.reshape(-1, 1) 
forcasted_output = scaler.inverse_transform(forcasted_output) 

In [ ]:
forcasted_output = pd.DataFrame(forcasted_output)
date = pd.DataFrame(time)
df_result = pd.concat([date,forcasted_output], axis=1)
df_result.columns = "Date", "Forecasted"
df_result

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Number Sold Forecasting For Next 60 Days')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Number Sold' ,fontsize=18)
plt.plot(s1p2['number_sold'])
plt.plot(df_result.set_index('Date')[['Forecasted']])

The next 60 days is not enough to show a good graph, so you can increase the time, but make sure that more needed days for forecasting, less accuracy will happen.

## 3rd Product

In [ ]:
# Store 1 with product 3
s1p3 = tr_df[(tr_df['store'] == 0) & (tr_df['product'] == 2)]
s1p3

In [ ]:
plt.figure(figsize = (15, 6))
plt.plot(s1p3.index, s1p3['number_sold'])
plt.xlabel('Date', {'fontsize': 12})
plt.ylabel('Number Sold', {'fontsize': 12})
plt.title('Number Sold Of Product 3 In The First Store')
plt.show()

In [ ]:
n_cols = 1
dataset = s1p3["number_sold"]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

In [ ]:
# 75% to Train , 25% to Test
train_size = int(len(data) * 0.75)
test_size = len(data) - train_size
print("Train Size :",train_size,"Test Size :",test_size)

train_data = scaled_data[0:train_size, :]
train_data.shape

In [ ]:
# Creating a Training set with 60 time-steps
x_train = []
y_train = []
time_steps = 60
n_cols = 1

for i in range(time_steps, len(train_data)):
    x_train.append(train_data[i-time_steps:i, :n_cols])
    y_train.append(train_data[i, :n_cols])
    if i<=time_steps:
        print('X_train: ', x_train)
        print('y_train:' , y_train)

In [ ]:
# Convert to numpy array
x_train, y_train = np.array(x_train), np.array(y_train)
# Reshaping the input to (n_samples, time_steps, n_feature)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_cols))
x_train.shape , y_train.shape

In [ ]:
model = Sequential([
    LSTM(50, return_sequences= True, input_shape= (x_train.shape[1], n_cols)),
    LSTM(64, return_sequences= False),
    Dense(32),
    Dense(16),
    Dense(n_cols)
])

model.compile(optimizer= 'adam', loss= 'mse' , metrics= "mean_absolute_error")
model.summary()

In [ ]:
# Fitting the LSTM to the Training set
callbacks = [EarlyStopping(monitor= 'loss', patience= 10 , restore_best_weights= True)]
history = model.fit(x_train, y_train, epochs= 100, batch_size= 32 , callbacks= callbacks )

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history["loss"])
plt.plot(history.history["mean_absolute_error"])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
# Creating a testing set with 60 time-steps and 1 output
time_steps = 60
test_data = scaled_data[train_size - time_steps:, :]

x_test = []
y_test = []
n_cols = 1

for i in range(time_steps, len(test_data)):
    x_test.append(test_data[i-time_steps:i, 0:n_cols])
    y_test.append(test_data[i, 0:n_cols])
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], n_cols))

x_test.shape , y_test.shape

In [ ]:
# Get Prediction
predictions = model.predict(x_test)
#inverse predictions scaling
predictions = scaler.inverse_transform(predictions)
predictions.shape

In [ ]:
#inverse y_test scaling
y_test = scaler.inverse_transform(y_test)

RMSE = np.sqrt(np.mean( y_test - predictions )**2).round(2)
RMSE

In [ ]:
preds_acts = pd.DataFrame(data={'Predictions':predictions.flatten(), 'Actuals':y_test.flatten()})
preds_acts

In [ ]:
train = dataset.iloc[:train_size , 0:1]
test = dataset.iloc[train_size: , 0:1]
test['Predictions'] = predictions

plt.figure(figsize= (16, 6))
plt.title('Number Sold Prediction', fontsize= 18)
plt.xlabel('Date', fontsize= 18)
plt.ylabel('Number Sold', fontsize= 18)
plt.plot(train['number_sold'], linewidth= 3)
plt.plot(test['number_sold'], linewidth= 3)
plt.plot(test["Predictions"], linewidth= 3)
plt.legend(['Train', 'Test', 'Predictions'])

In [ ]:
from datetime import timedelta

In [ ]:
def insert_end(Xin, new_input):
    timestep = 60
    for i in range(timestep - 1):
        Xin[:, i, :] = Xin[:, i+1, :]
    Xin[:, timestep - 1, :] = new_input
    return Xin

In [ ]:
future = 30 * 2
forcast = []
Xin = x_test[-1 :, :, :]
time = []
for i in range(0, future):
    out = model.predict(Xin, batch_size=5)
    forcast.append(out[0, 0]) 
    print(forcast)
    Xin = insert_end(Xin, out[0, 0]) 
    time.append(pd.to_datetime(s1p1.index[-1]) + timedelta(days=i))

In [ ]:
time

In [ ]:
forcasted_output = np.asanyarray(forcast)   
forcasted_output = forcasted_output.reshape(-1, 1) 
forcasted_output = scaler.inverse_transform(forcasted_output) 

In [ ]:
forcasted_output = pd.DataFrame(forcasted_output)
date = pd.DataFrame(time)
df_result = pd.concat([date,forcasted_output], axis=1)
df_result.columns = "Date", "Forecasted"
df_result

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Number Sold Forecasting For Next 60 Days')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Number Sold' ,fontsize=18)
plt.plot(s1p3['number_sold'])
plt.plot(df_result.set_index('Date')[['Forecasted']])

## It will take a time to try all the data ! If you want to do that, you won't find any error but take care when you chose the product data..

## If you find this notebook make sense, please upvote it and follow me.
## THANK YOU..